# Notebook 5: Evaluation / Evaluación

In this notebook, we will compare the ML pipeline from notebook 3 with the Deep Learning pipeline from notebook 4.
 <hr style="border:2px solid gray">

En este Jupyter notebook, compararemos el proceso de aprendizaje automático del Notebook 3 con el proceso de aprendizaje profundo del Notebook 4.

---

## Ensuring Fair Evaluation / Garantizar una evaluación justa

This code sets the CPU affinity to core 0, ensuring that all threads and processes are restricted to a single CPU core. By doing so, you ensure consistent performance evaluation, as it prevents interference from other cores and provides a controlled environment for comparing methods on the same core.
 <hr style="border:2px solid gray">

Este código establece la afinidad de la CPU en el núcleo 0, asegurando que todos los hilos y procesos están restringidos a un único núcleo de la CPU. De este modo, se garantiza una evaluación coherente del rendimiento, ya que se evitan las interferencias de otros núcleos y se proporciona un entorno controlado para comparar métodos en el mismo núcleo.


In [ ]:
import psutil
import os

# Set CPU affinity to core 0 (the first core)
p = psutil.Process(os.getpid())
p.cpu_affinity([0])

---

## Pipeline Evaluation / Evaluación de la cadena de procesamiento

In this step, we are developing an evaluation strategy that aims to provide a comprehensive assessment of a given machine-learning pipeline by combining both performance metrics and resource usage metrics. Here’s what we’re trying to achieve:

1. **Performance Metrics Evaluation**:
    - **Accuracy and F1 Score**: Measure the model’s ability to correctly classify images.
	- **Confusion Matrix Analysis**: Identify specific classes where the model may be underperforming.
<br>
<br>



2. **Resource Utilization Assessment**:
	- **Evaluation Time**: Determine the total time taken for preprocessing and prediction.
	- **Pipeline Size**: Calculate the combined size of the trained model and preprocessing steps to understand storage requirements.
	- **Memory Consumption**: Monitor peak memory usage during evaluation to ensure it fits within hardware constraints.
	- **CPU Usage**: Measure average CPU utilization to evaluate computational efficiency.


    The primary objective is to not only achieve high classification accuracy but also to assess and optimize the pipeline’s computational efficiency and resource utilization. This holistic approach ensures that the model is not just effective but also practical for deployment, especially in environments with limited computational resources.


 <hr style="border:2px solid gray">


En este paso, estamos desarrollando una estrategia de evaluación que tiene como objetivo proporcionar una evaluación completa de una determinada canalización de aprendizaje automático mediante la combinación de métricas de rendimiento y métricas de uso de recursos. Esto es lo que intentamos conseguir:


1. **Evaluación de las métricas de rendimiento**:
    - **Precisión y puntuación F1**: Mide la capacidad del modelo para clasificar correctamente las imágenes.
	- **Análisis de la matriz de confusión**: Identificar clases específicas en las que el modelo puede estar rindiendo por debajo de lo esperado.
<br>
<br>



2. **Evaluación de la utilización de recursos**:
	- **Tiempo de evaluación**: Determinar el tiempo total empleado en el preprocesamiento y la predicción.
	- **Tamaño del modelo**: Calcular el tamaño combinado del modelo entrenado y los pasos de preprocesamiento para comprender los requisitos de almacenamiento.
	- **Consumo de memoria**: Supervise el uso máximo de memoria durante la evaluación para asegurarse de que se ajusta a las limitaciones del hardware.
	- **Uso de la CPU**: Mida la utilización media de la CPU para evaluar la eficiencia computacional.


El objetivo principal no es sólo lograr una alta precisión de clasificación, sino también evaluar y optimizar la eficiencia computacional y la utilización de recursos de la tubería. Este enfoque holístico garantiza que el modelo no solo sea eficaz, sino también práctico para su despliegue, especialmente en entornos con recursos computacionales limitados.



In [ ]:
from source.pre import evaluate_pipeline # A built-in function to evaluate a given ML pipeline by preprocessing, predicting, and calculating performance metrics.

**Inputs:**
- **model**: The trained machine learning model to evaluate.
- **X_test_raw**: Raw test data that needs to be preprocessed before evaluation.
- **y_test**: True labels corresponding to the test data for performance comparison.
- **preprocessing_fn**: A function used to preprocess the raw test data.
    
**Outputs:**
- **metrics**: A dictionary containing various evaluation metrics like accuracy, F1 score, evaluation time, memory usage, CPU usage, and pipeline size.


 <hr style="border:2px solid gray">





**Entradas:**
- **model**: El modelo de aprendizaje automático entrenado para evaluar.
- **X_test_raw**: Datos de prueba sin procesar que necesitan ser preprocesados antes de la evaluación.
- **y_test**: Etiquetas verdaderas correspondientes a los datos de prueba para comparar el rendimiento.
- **preprocessing_fn**: Función utilizada para preprocesar los datos de prueba sin procesar.
    
**Salidas:**
- **metrics**: Un diccionario que contiene varias métricas de evaluación como precisión, puntuación F1, tiempo de evaluación, uso de memoria, uso de CPU y tamaño del pipeline.


---

### Preprocessing (Testing data) / Preprocesamiento (datos de prueba)

By applying the same preprocessing to both the training and testing data, we ensure consistent feature representation, which is essential for accurate predictions and prevents errors from data mismatches. This alignment improves model evaluation and generalization to new data. Additionally, combining all preprocessing steps into a single function allows the evaluation function to track execution time and resource usage, ensuring both consistency and computational efficiency across the entire pipeline.

 <hr style="border:2px solid gray">


Al aplicar el mismo preprocesamiento tanto a los datos de entrenamiento como a los de prueba, garantizamos una representación coherente de las características, lo que resulta esencial para realizar predicciones precisas y evita errores derivados de la falta de coincidencia de los datos. Esta alineación mejora la evaluación del modelo y la generalización a nuevos datos. Además, la combinación de todos los pasos de preprocesamiento en una única función permite que la función de evaluación controle el tiempo de ejecución y el uso de recursos, garantizando la coherencia y la eficiencia computacional en todo el proceso.

### ML method Preprocessing / Método de aprendizaje automático Preprocesamiento

In [ ]:
def preprocessing_fn_ML(X): # We apply the same pre-processing steps implemented in Notebook 3.
    from skimage.color import rgb2gray
    from skimage.transform import resize
    
    # Normalize the data to [0, 1]
    X_pre = X.astype('float32') / 255.0
    
    # Convert to grayscale
    X_pre = np.array([rgb2gray(image) for image in X_pre])
    
    # Resize images to 64x64 pixels
    X_pre = np.array([resize(image, (64, 64), anti_aliasing=True) for image in X_pre])
    
    # Flatten the images
    num_samples = X_pre.shape[0]
    X_pre = X_pre.reshape(num_samples, -1)
    
    return X_pre

---

### ML Evaluation / Evaluación del aprendizaje automático

#### Import data and ML model /  Importar datos y modelo de aprendizaje automático

In [ ]:
import numpy as np
# first let us load the testing data
test_images = np.load('test_images.npy')      # Load image test data
test_labels = np.load('test_labels.npy')      # Load label test data

In [ ]:
import pickle
# Load the ml model from the 3rd notebook
with open('sgd_model.pkl', 'rb') as file:
    sgd_model = pickle.load(file)

In [ ]:
# Assuming you have:
# - A trained model named like 'lr_model'
# - Raw test data 'X_test_raw'
# - True labels 'y_test'
# - All pre-processing methods gathered in one function



# Evaluate the pipeline
metrics = evaluate_pipeline(sgd_model, test_images, test_labels, preprocessing_fn_ML)

# Print the evaluation metrics
print("Evaluation Metrics:")
for key, value in metrics.items():
    if key == 'evaluation_time':
        print(f"{key}: {value:.2f} seconds")
    elif key == 'pipeline_size':
        print(f"{key}: {value:.2f} MB")
    elif key == 'peak_memory_usage':
        print(f"{key}: {value:.2f} MB")
    elif key == 'average_cpu_usage':
        print(f"{key}: {value:.2f}%")
    elif key == 'confusion_matrix':
        print(key)
        print(value) 
    else:
        print(f"{key}: {value:.4f}")

---

### CubeSatNet_CNN Evaluation / Evaluación  del CubeSatNet_CNN

#### preprocessing / preprocesamiento

In [ ]:
def preprocessing_fn_CNN(X):  # we did not use any preprocessing in notebook 4
    
    return X

#### Import CNN model / Importar modelo CNN

In [ ]:
# Load the CNN model from the 4th notebook
with open('cnn_model.pkl', 'rb') as file:
    cnn_model = pickle.load(file)

#### Evaluate the CNN pipeline /  Evaluar el CNN

In [ ]:
from keras.utils import to_categorical
test_labels = to_categorical(test_labels, num_classes=5)


# Evaluate the pipeline
metrics = evaluate_pipeline(cnn_model, test_images, test_labels, preprocessing_fn_CNN)

# Print the evaluation metrics
print("Evaluation Metrics:")
for key, value in metrics.items():
    if key == 'evaluation_time':
        print(f"{key}: {value:.2f} seconds")
    elif key == 'pipeline_size':
        print(f"{key}: {value:.2f} MB")
    elif key == 'peak_memory_usage':
        print(f"{key}: {value:.2f} MB")
    elif key == 'average_cpu_usage':
        print(f"{key}: {value:.2f}%")
    elif key == 'confusion_matrix':
        print(key)
        print(value) 
    else:
        print(f"{key}: {value:.4f}")

---

### Comparison between the two models / Comparación entre los dos modelos